In [1]:
import julia
julia.install()

from pathlib import Path

# # Point to the top of the project relative to this script
def projectdir(*args):
    return str(Path.cwd().joinpath("..", "..", "..", *args).resolve())

def print_allocated_memory():
   print("{:.2f} GB".format(torch.cuda.memory_allocated() / 1024 ** 3))

In [2]:
from torch.utils.data import DataLoader
from julia import Main as jl
from sklearn.metrics import accuracy_score
import ipdb
import torch
# from torchvision.transforms import Lambda
from tqdm import tqdm

from torchvision.models import resnet50
from torchvision.models import ResNet50_Weights
from torchvision import models
from torchvision.models.feature_extraction import create_feature_extractor

class DDVFAStrategy():
    """DDVFA Strategy"""

    def __init__(self):
        jl.project_dir = projectdir()
        jl.eval("using Pkg; Pkg.activate(project_dir)")
        jl.eval("using AdaptiveResonance")
        jl.eval("art = DDVFA(rho_lb=0.4, rho_ub=0.75)")
        # jl.eval("art.config = DataConfig(0.0, 255.0, 28*28)")
        jl.eval("art.config = DataConfig(0, 1.0, 49)")
        
        rn = resnet50()
        self.mod = create_feature_extractor(rn, {'layer4': 'layer4'})
        self.mod = self.mod.to('cuda')
        self.mod.eval()
        self.weights = ResNet50_Weights.DEFAULT
        self.preprocess = self.weights.transforms()
        self.min = 10.0
        self.max = 70.0

    def ext_features(self, img):
        # print(rgbimg.shape)
        # prep = self.preprocess(rgbimg)
        img = img.to('cuda')
        prep = self.preprocess(img)
        features = self.mod(prep)['layer4']
        # avg_features = features.mean(dim=1).flatten(start_dim=1).detach().cpu().numpy()
        avg_features = features.detach().mean(dim=1).flatten(start_dim=1)
        avg_features = (avg_features - self.min) / (self.max - self.min) * 2 - 1
        avg_features = avg_features.sigmoid().cpu().numpy()
        # avg_features = features.mean(dim=1).flatten(start_dim=1).cpu().numpy()
        # ipdb.set_trace()
        # avg_features.flatten().detach().numpy()
        return avg_features

    def train(self, experience):
        train_dataset = experience.dataset
        t = experience.task_label
        train_data_loader = DataLoader(
            # train_dataset,
            dataset=train_dataset,
            pin_memory=True,
            # num_workers=1,
            # num_workers=4,
            # batch_size=64,
            batch_size=90,
            # transform=self.trans
        )
        print(experience.dataset.__len__())
        for mb in tqdm(train_data_loader):
            # ld = mb[0].to('cuda')
            data, labels, tasks = mb
            af = self.ext_features(data)
            jl.features = af
            jl.labels = labels.numpy()
            # ipdb.set_trace()
            jl.eval("train!(art, features, y=labels)")
            # for sample, label, task in mb:
            # for sample in mb:
            #     self.ext_features(sample)

        # for sample in train_dataset:
        # n_samples = len(train_dataset)
        # for ix in range(n_samples):
        # for ix in tqdm(range(n_samples)):
            # ls, ll, lt = train_dataset[ix]
            # self.ext_features(ls)
            # self.model(ls)

            # # ipdb.set_trace()
            # jl.sample = ls.flatten().numpy()
            # jl.label = ll
            # jl.t = lt
            # jl.eval("train!(art, sample, y=label)")

    def eval(self, experience):
        eval_dataset = experience.dataset
        t = experience.task_label
        # eval_data_loader = DataLoader(
        #     # eval_dataset, num_workers=4, batch_size=128
        #     eval_dataset, num_workers=4, batch_size=32
        # )
        # print(eval_data_loader)

        total = 0
        correct = 0
        # for mb in eval_data_loader:
        #     for sample, label in mb:
        #         jl.sample = sample.transpose()
        #         jl.label = label.numpy()
        #         y_hat = jl.eval("classify(art, sample)")

        eval_data_loader = DataLoader(
            # train_dataset,
            dataset=train_dataset,
            pin_memory=True,
            # num_workers=1,
            # num_workers=4,
            # batch_size=64,
            batch_size=90,
            # transform=self.trans
        )
        print(experience.dataset.__len__())
        for mb in tqdm(train_data_loader):
            # ld = mb[0].to('cuda')
            data, labels, tasks = mb
            af = self.ext_features(data)
            jl.features = af
            jl.labels = labels.numpy()
            # ipdb.set_trace()
            y_hats = jl.eval("train!(art, features, y=labels)")
            correct = torch.sum(y_hats == labels)

            # total += 1
            # if y_hat == label:
            #     correct += 1
        
        return correct/total
        # j.samples = mb
        # y_hats = j.eval("classify(art, samples)")
        # accuracy_score(y_test, y_hat)

print_allocated_memory()

0.00 GB


In [3]:
from avalanche.benchmarks import SplitMNIST
from avalanche.benchmarks.classic import SplitCIFAR10
from torchvision.transforms import Lambda

trans = Lambda(lambda x: x.repeat(3, 1, 1) if x.size(0)==1 else x)
# Benchmark creation
benchmark = SplitMNIST(
    n_experiences=5,
    train_transform=trans,
    eval_transform=trans,
)
# benchmark = SplitCIFAR10(n_experiences=5, return_task_id=True)

# Create the Strategy Instance
cl_strategy = DDVFAStrategy()

# Training Loop
print('Starting experiment...')

for exp_id, experience in enumerate(benchmark.train_stream):
    print("Start of experience ", experience.current_experience)

    # print(experience.dataset)
    cl_strategy.train(experience)
    print('Training completed')

    print('Computing accuracy on the current test set')
    cl_strategy.eval(benchmark.test_stream[exp_id])
# features.mean(dim=1).flatten(start_dim=1).detach().cpu().numpy()
# features.mean(dim=1).flatten(start_dim=1)
# def lin_normalize()
# min = 10
# max = 70


Starting experiment...
Start of experience  0
12188


 51%|█████▏    | 70/136 [00:20<00:19,  3.39it/s]


KeyboardInterrupt: 

In [ ]:
b = torch.rand((2, 3), dtype=torch.float64) * 20.
print(b)
b - 10

tensor([[15.9900,  3.4863,  7.3100],
        [ 4.1277,  1.6839, 16.4035]], dtype=torch.float64)


tensor([[ 5.9900, -6.5137, -2.6900],
        [-5.8723, -8.3161,  6.4035]], dtype=torch.float64)

In [ ]:
inp = torch.randn(2, 3, 224, 224)
model(inp)

NameError: name 'model' is not defined